# 🔗 ASCON Hash Functions Implementation

[![Python](https://img.shields.io/badge/Python-3.7+-blue.svg)](https://python.org)
[![License](https://img.shields.io/badge/License-MIT-green.svg)](LICENSE)
[![NIST](https://img.shields.io/badge/NIST-Lightweight%20Cryptography-red.svg)](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-232.pdf)

A complete Python implementation of the **ASCON Hash** family of cryptographic hash functions, part of the NIST Lightweight Cryptography standard. This implementation provides three variants of ASCON hash functions with different output capabilities.

## 🌟 Overview

ASCON Hash is part of the ASCON family of lightweight cryptographic primitives, selected as the winner of the NIST Lightweight Cryptography competition. This implementation includes:

- 🏷️ **ASCON-Hash-256**: Fixed 256-bit output hash function
- 🔄 **ASCON-Hash-XOF-128**: Extendable Output Function with variable length
- 🎯 **ASCON-Hash-CXOF-128**: Customizable XOF with additional context input


## 🔍 Algorithm Details

### Common Features
All ASCON hash functions share these characteristics:
- **State Size**: 320 bits (5 × 64-bit words)
- **Permutation Rounds**: 12 rounds for all operations
- **Block Size**: 64 bits (rate for absorption)
- **Security Level**: 128 bits

## 🎯 Use Cases

### ASCON-Hash-256 Applications
- 📄 **Document Integrity**: File checksums and verification
- 🔐 **Password Hashing**: With proper salt and iterations
- 🔗 **Blockchain**: Block header hashing
- 📱 **IoT Security**: Lightweight device authentication

### XOF Applications
- 🗝️ **Key Derivation**: Generate multiple keys from master secret
- 🎲 **Pseudorandom Generation**: Deterministic random bit streams
- 🔄 **Stream Ciphers**: Keystream generation
- 📊 **Testing**: Generate test vectors of arbitrary length

### CXOF Applications
- 🏷️ **Domain Separation**: Different contexts for same input
- 🔧 **Protocol Versioning**: Hash function customization
- 🎨 **Application-Specific**: Custom hash functions per use case
- 🛡️ **Security Protocols**: Context-aware cryptographic operations


### Advantages
- 💪 **Lightweight**: Optimized for resource-constrained environments
- ⚡ **Fast**: Efficient permutation-based design
- 🛡️ **Secure**: NIST-approved algorithm with 128-bit security
- 🔧 **Flexible**: Multiple variants for different needs
- 🎯 **Versatile**: Fixed and variable output lengths


## 🧮 Implementation Details

### Padding Scheme
ASCON uses a simple padding scheme:
- Append '1' bit to message
- Pad with '0' bits to reach block boundary
- Formula: `message + '1' + '0'^j` where `j = (-len(message)-1) mod rate`

### State Structure
```
State: [X0|X1|X2|X3|X4] (5 × 64-bit words)
Rate: 64 bits (X0 for absorption/squeezing)
Capacity: 256 bits (X1-X4 for security)
```

### Round Function
Each round consists of:
1. **AddRoundConstant**: XOR constant into X2
2. **SubBytes**: Apply 5-bit S-box to each bit position
3. **ShiftRows**: Rotate each word by fixed amounts

## 🔄 Comparison with Other Hash Functions

| Feature | ASCON-Hash | SHA-256 | SHA-3 | BLAKE2 |
|---------|------------|---------|-------|---------|
| **Output Size** | 256-bit/Variable | 256-bit | Variable | Variable |
| **Security Level** | 128-bit | 128-bit | Variable | Variable |
| **Lightweight** | ✅ Excellent | ❌ No | ⚠️ Moderate | ⚠️ Moderate |
| **IoT Suitable** | ✅ Yes | ❌ No | ⚠️ Limited | ✅ Yes |
| **Standardized** | ✅ NIST LWC | ✅ NIST FIPS | ✅ NIST FIPS | ❌ No |
| **Variable Output** | ✅ XOF/CXOF | ❌ No | ✅ SHAKE | ✅ Yes |


## References

📚 **Official NIST Documentation:**
- [Link](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-232.pdf)


In [9]:
def str_to_binstring(s: str) -> str:
    """Convert a text string into a binary string"""
    return "".join(f"{ord(c):08b}" for c in s)

def binstring_to_str(b: str) -> str:
    """Convert a binary string back to the original text string."""
    if len(b) % 8 != 0:
        raise ValueError("Invalid binary string length: must be a multiple of 8")
    return "".join(chr(int(b[i:i+8], 2)) for i in range(0, len(b), 8))

In [10]:
def parse(X, r):
    l = len(X) / r
    x = ["0"] * (int(l) + 1)
    for i in range(int(l)):
        x[i] = X[i * r : ((i + 1) * (r))]
    x[int(l)] = X[int(l) * r : len(X)]
    return x

def pad(X, r):
    """pads according to ascon hash"""
    j = (-len(X) - 1) % r
    return X + "1" + "0" * j

def bit_xor(a: str, b: str) -> str:
    """XOR two bit strings of equal length"""
    # if len(a) != len(b):
    # raise ValueError(f"Bit string lengths don't match: {len(a)} vs {len(b)}")
    return "".join("1" if x != y else "0" for x, y in zip(a, b))

def right_rotation(X: str, i: int) -> str:
    """Right rotation by i bits of a 64-bit word X"""
    X = X.zfill(64)  # Ensure 64 bits
    i %= 64
    return X[-i:] + X[:-i] if i > 0 else X

def compute_sbox_bit_slice(x0, x1, x2, x3, x4):
    # Apply S-box bit by bit
    result = [[], [], [], [], []]
    for i in range(64):
        # Extract bits at position i
        bits = [int(x0[i]), int(x1[i]), int(x2[i]), int(x3[i]), int(x4[i])]
        y0 = str(
            (bits[4] & bits[1])
            ^ bits[3]
            ^ (bits[2] & bits[1])
            ^ bits[2]
            ^ (bits[1] & bits[0])
            ^ bits[1]
            ^ bits[0]
        )
        y1 = str(
            bits[4]
            ^ (bits[3] & bits[2])
            ^ (bits[3] & bits[1])
            ^ bits[3]
            ^ (bits[2] & bits[1])
            ^ bits[2]
            ^ bits[1]
            ^ bits[0]
        )
        y2 = str((bits[4] & bits[3]) ^ bits[4] ^ bits[2] ^ bits[1] ^ 1)
        y3 = str(
            (bits[4] & bits[0])
            ^ bits[4]
            ^ (bits[3] & bits[0])
            ^ bits[3]
            ^ bits[2]
            ^ bits[1]
            ^ bits[0]
        )
        y4 = str(
            (bits[4] & bits[1]) ^ bits[4] ^ bits[3] ^ (bits[1] & bits[0]) ^ bits[1]
        )

        result[0].append(y0)
        result[1].append(y1)
        result[2].append(y2)
        result[3].append(y3)
        result[4].append(y4)

    return ["".join(result[i]) for i in range(5)]

const_i = [
    0x000000000000003C,
    0x000000000000002D,
    0x000000000000001E,
    0x000000000000000F,
    0x00000000000000F0,
    0x00000000000000E1,
    0x00000000000000D2,
    0x00000000000000C3,
    0x00000000000000B4,
    0x00000000000000A5,
    0x0000000000000096,
    0x0000000000000087,
    0x0000000000000078,
    0x0000000000000069,
    0x000000000000005A,
    0x000000000000004B,
]

def pho_c(S, rnd, round_num):
    """Add round constant to S2"""
    # Extract words
    words = [S[i * 64 : (i + 1) * 64] for i in range(5)]
    # Round constant
    const_idx = 16 - rnd + round_num
    const_bits = bin(const_i[const_idx])[2:].zfill(64)
    # XOR into S2
    words[2] = bit_xor(words[2], const_bits)
    return "".join(words)

def pho_s(S):
    """Apply S-box layer"""
    words = [S[i * 64 : (i + 1) * 64] for i in range(5)]
    words = compute_sbox_bit_slice(*words)
    return "".join(words)

def pho_l(S):
    """Apply linear diffusion layer"""
    words = [S[i * 64 : (i + 1) * 64] for i in range(5)]

    words[0] = bit_xor(
        words[0], bit_xor(right_rotation(words[0], 19), right_rotation(words[0], 28))
    )
    words[1] = bit_xor(
        words[1], bit_xor(right_rotation(words[1], 61), right_rotation(words[1], 39))
    )
    words[2] = bit_xor(
        words[2], bit_xor(right_rotation(words[2], 1), right_rotation(words[2], 6))
    )
    words[3] = bit_xor(
        words[3], bit_xor(right_rotation(words[3], 10), right_rotation(words[3], 17))
    )
    words[4] = bit_xor(
        words[4], bit_xor(right_rotation(words[4], 7), right_rotation(words[4], 41))
    )

    return "".join(words)

def ascon_round(S, rnd, round_num):
    """One round = AddConst -> S-box -> Linear"""
    S = pho_c(S, rnd, round_num)
    S = pho_s(S)
    S = pho_l(S)
    return S

def ascon_permutation(S, rounds):
    """Full Ascon permutation"""
    S = S.zfill(320)
    for round_num in range(rounds):
        S = ascon_round(S, rounds, round_num)
    return S

In [11]:
def bin_to_hex(b: str) -> str:
    """Convert a binary string to hexadecimal string."""
    b = b.zfill((len(b) + 3) // 4 * 4)
    return hex(int(b, 2))[2:]

In [12]:
def Ascon_Hash_256(M):
    const_IV = "0000000000000000000010000000000100000000110011000000000000000010"
    # print(len(const_IV))
    S = ascon_permutation(const_IV + "0" * 256, 12)
    message_bit = str_to_binstring(M)
    blocks = parse(message_bit, 64)
    blocks[len(blocks) - 1] = pad(blocks[len(blocks) - 1], 64)
    for i in range(len(blocks)):
        S = bit_xor(S[0:64], blocks[i]) + S[64:]
        S = ascon_permutation(S, 12)
    S = bit_xor(S[0:64], blocks[len(blocks) - 1]) + S[64:]
    S = ascon_permutation(S, 12)
    H = []
    for i in range(3):
        H.append(S[0:64])
        S = ascon_permutation(S, 12)
    H.append(S[0:64])
    return "".join(H)
plaintext = "Hello, World! this is a test message for ASCON-Hash-256."
a1 = Ascon_Hash_256(plaintext)
print(f"ASCON-Hash-256('{plaintext}') = {bin_to_hex(a1)}")

ASCON-Hash-256('Hello, World! this is a test message for ASCON-Hash-256.') = a5147ceb7ed324677ebd026b4c2ccdf0496d22d94e3c1a782c69d29973f104f9


In [13]:
import math
def Ascon_hash_XOF_128(M, L):
    const_IV = "0000000000000000000010000000000000000000110011000000000000000011"
    S = ascon_permutation(const_IV + "0" * 256, 12)
    message_bits = str_to_binstring(M)
    blocks = parse(message_bits, 64)
    blocks[len(blocks) - 1] = pad(blocks[len(blocks) - 1], 64)
    for i in range(len(blocks)):
        S = bit_xor(S[0:64], blocks[i]) + S[64:]
        S = ascon_permutation(S, 12)
    S = bit_xor(S[0:64], blocks[len(blocks) - 1]) + S[64:]
    S = ascon_permutation(S, 12)
    h = math.ceil(L / 64) - 1
    H = []
    for i in range(h):
        H.append(S[0:64])
        S = ascon_permutation(S, 12)
    H.append(S[0:64])
    H_dash = "".join(H)
    return H_dash[0:L]

plaintext="This is a test message for ASCON-Hash-XOF-128."
a2 = Ascon_hash_XOF_128(plaintext, 512)
print(f"ASCON-Hash-XOF-128('{plaintext}', 1024) = {bin_to_hex(a2)}")

ASCON-Hash-XOF-128('This is a test message for ASCON-Hash-XOF-128.', 1024) = 75b2d12f05197bab36ec353001def958b42981875acb8dba0ebf9cd637ce6499f13d0c6b0994a245639012456403d58b941eeee06e6ffb1d748ac8daded9ff66


In [14]:
def int_to_int64_binstring(x: int) -> str:
    """Convert integer x into a 64-bit binary string."""
    return format(x, "064b")


def Ascon_hash_CXOF_128(M, L, Z):
    z_bits = str_to_binstring(Z)
    if len(z_bits) > 2048:
        return "Z should be less than equal to 2048 bits"
    const_IV = "0000000000000000000010000000000000000000110011000000000000000100"
    S = ascon_permutation(const_IV + "0" * 256, 12)
    Z0 = int_to_int64_binstring(len(Z))
    z_blocks = parse(z_bits, 64)
    message_bits = str_to_binstring(M)
    z_temp = Z0 + "".join(z_blocks) + message_bits
    z_blocks[len(z_blocks) - 1] = pad(z_blocks[len(z_blocks) - 1], 64)
    for i in range(len(z_blocks)):
        S = bit_xor(S[0:64], z_blocks[i]) + S[64:]
        S = ascon_permutation(S, 12)
    m_blocks = parse(z_temp, 64)
    m_blocks[len(m_blocks) - 1] = pad(m_blocks[len(m_blocks) - 1], 64)
    for i in range(len(m_blocks)):
        S = bit_xor(S[0:64], m_blocks[i]) + S[64:]
        S = ascon_permutation(S, 12)
    S = bit_xor(S[0:64], m_blocks[len(m_blocks) - 1]) + S[64:]
    S = ascon_permutation(S, 12)
    h = math.ceil(L / 64) - 1
    H = []
    for i in range(h):
        H.append(S[0:64])
        S = ascon_permutation(S, 12)
    H.append(S[0:64])
    H_dash = "".join(H)
    return H_dash[0:L]


a33 = Ascon_hash_CXOF_128(
    "This is a test message for ASCON-Hash-CXOF-128", 512, "Test Message"
)
print(f"ASCON-Hash-CXOF-128 = {bin_to_hex(a33)}")

ASCON-Hash-CXOF-128 = 3b846dac76ca678d4cffd3598fd5da5eb325fa556460fea5cd9eb0b1a1bab2eced98127ce878bd2d84494f065c90c380ba4506538c179934f0671bea33780ef8
